In [2]:
import face_recognition
import numpy as np
import glob
import json
from flask import Flask, request, jsonify, render_template
import os
# from flask_httpauth import HTTPBasicAuth
from sys import getsizeof
from PIL import Image
from io import BytesIO
# from flask_jwt_extended import create_access_token, JWTManager, jwt_required, get_jwt_identity
from flask_cors import CORS
import pickle
app = Flask(__name__)
CORS(app)
ALLOWED_EXTENSIONS = ['png','jpg']
UPLOAD_FOLDER = 'images'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def load_link_url(link_to_face):
    data_txt = glob.glob('data/video/*.txt')
    for filename_face in link_to_face:
        file_text = '.'.join(filename_face.split('.')[:-1])+'.txt'
#         print(file_text)
        if file_text not in data_txt:
            link_to_face[filename_face]['url'] = 'not_found'
        else:
            link_to_face[filename_face]['url'] = json.load(open(file_text,'r'))['url']
    return link_to_face
def iterate_face(img, threshold=0.4):
    poi_face = face_recognition.face_encodings(np.array(img))
    found_data = []
    for key in all_faces.keys():
        for face in all_faces[key]['agg_faces'].keys():
            distance = face_recognition.face_distance(all_faces[key]['agg_faces'][face]['face_image'],poi_face)
            if distance<threshold:
                found_data.append((key,face))
    return found_data

def find_face_on_data(img):
    res = iterate_face(img)
    all_aggregate_face = {}
    for i in res:
#         print(i[0])
        if i[0] in all_aggregate_face.keys():
            all_aggregate_face[i[0]]['time']+=list(np.array(all_faces[i[0]]['agg_faces'][i[1]]['time'])*10)
        else:
            all_aggregate_face[i[0]] = {}
            all_aggregate_face[i[0]]['url']=all_faces[i[0]]['url']
            all_aggregate_face[i[0]]['time']=list(np.array(all_faces[i[0]]['agg_faces'][i[1]]['time'])*10)
    return all_aggregate_face


all_faces = pickle.load(open('face_features_extracted','rb'))
all_faces = load_link_url(all_faces)

In [167]:
import io
import re
import glob
import nltk
import numpy as np
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer 
nltk.download('stopwords')

def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.dot(a, a.T)*np.dot(b.T, b))
    return cos_sim
def cosine_sim_batch(a,b):
    similarity = a * b
    similarity = similarity.toarray()
    return similarity
class Retrieval :
    def __init__(self) :
        self.stop_words = set(stopwords.words('indonesian')) 
        self.stop_words.add('telkom')
        self.stop_words.add('indonesia')
                
    def retrieve(self, query, n, ngram=1):
        if ngram == 2 :
            df = self.retrieve_bigram(query)
        else :
            df = self.retrieve_unigram(query)
        
#         files = get_files()
#         df.docs = df.docs.str.replace('.txt', '.pdf')

#         df = pd.merge(df, files, on='docs')
        
        df = df.head(n)
        df['file'] = df.docs.apply(lambda x: x.split('_&_')[0])
        df['words'] = df.docs.apply(lambda x: x.split('_&_')[1])
        df = df.drop('docs',axis=1)
        return df
    
    def retrieve_unigram(self, query) :
        query_vector_unigram = self.unigram.transform([query])
        
        scores_unigram = cosine_sim_batch(self.mentahan_tfidf_unigram,query_vector_unigram.T).ravel()
                
        self.result_unigram = scores_unigram
        df = pd.DataFrame({'docs':self.vectors_unigram.keys(), 'score':scores_unigram})
        df = df.sort_values('score', ascending=False).reset_index(drop=True)
        return df
    
    def retrieve_bigram(self, query) :
        query_vector_bigram = self.bigram.transform([query])
        scores_bigram = []
        for f in self.vectors_bigram :
            a = self.vectors_bigram[f]
            b = query_vector_bigram.T
            s = cosine_sim(a, b)
            scores_bigram.append(s.A1[0])
        self.result_bigram = scores_bigram
        df = pd.DataFrame({'docs':selfself.vectors_bigram.keys(), 'score':scores_bigram})
        df = df.sort_values('score', ascending=False).reset_index(drop=True)
        return df

    def retrieve_unigram_depricated(self, query) :
        query_vector_unigram = self.unigram.transform([query])
        scores_unigram = []
        for f in self.vectors_unigram :
            a = self.vectors_unigram[f]
            b = query_vector_unigram.T
            s = cosine_sim(a, b)
            scores_unigram.append(s.A1[0])
                
        self.result_bigram = scores_unigram
        df = pd.DataFrame({'docs':self.vectors_unigram.keys(), 'score':scores_unigram})
        df = df.sort_values('score', ascending=False).reset_index(drop=True)
        return df
    
    def retrieve_bigram_depricated(self, query) :
        query_vector_bigram = self.bigram.transform([query])
        scores_bigram = []
        for f in self.vectors_bigram :
            a = self.vectors_bigram[f]
            b = query_vector_bigram.T
            s = cosine_sim(a, b)
            scores_bigram.append(s.A1[0])
        self.result_bigram = scores_bigram
        df = pd.DataFrame({'docs':self.vectors_bigram.keys(), 'score':scores_bigram})
        df = df.sort_values('score', ascending=False).reset_index(drop=True)
        return df

    def sync(self, all_data) :
        bigram=TfidfVectorizer(use_idf=True, stop_words=self.stop_words, ngram_range=(1, 2))
        unigram=TfidfVectorizer(use_idf=True, stop_words=self.stop_words)
        symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
        lines = {}
        list_sentence = []
        
        for f in all_data.keys():
            for sentence in all_data[f]['agg_text'].keys():
                lines[f+'_&_'+sentence] = sentence
                list_sentence.append(sentence)
                
        tfidf_bigram = bigram.fit_transform(lines.values())
        tfidf_unigram = unigram.fit_transform(lines.values())                
        
        self.mentahan_tfidf_unigram = tfidf_unigram
        self.mentahan_tfidf_bigram = tfidf_bigram

        vectors_bigram = {f:v for f, v in zip(lines.keys(), tfidf_bigram)}
        vectors_unigram = {f:v for f, v in zip(lines.keys(), tfidf_unigram)}
        
        self.list_sentence = list_sentence
        self.bigram = bigram
        self.unigram = unigram
        self.vectors_bigram = vectors_bigram
        self.vectors_unigram = vectors_unigram
        

[nltk_data] Downloading package stopwords to /home/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [214]:
retri = Retrieval()
retri.sync(all_faces)

In [173]:
result.li

['score', 'file', 'words']

In [219]:
def query_search(txt,obj_retri,find_on_this):
    result = obj_retri.retrieve(txt,n=20)
    all_result = {}
    for i in result.T.to_dict().values():
        if i['file'] not in all_result.keys():
            all_result[i['file']] = {}
            all_result[i['file']]['time'] = find_on_this[i['file']]['agg_text'][i['words']]
            all_result[i['file']]['url'] = find_on_this[i['file']]['url']
        else:
            all_result[i['file']]['time'] += find_on_this[i['file']]['agg_text'][i['words']]
    return all_result

In [220]:
query_search('android',retri,all_faces)

{'data/video/How to Increase Code Quality with Clean Code on Android.mp4': {'time': [{'start': 26,
    'end': 26},
   {'start': 130, 'end': 130},
   {'start': 131, 'end': 131},
   {'start': 11, 'end': 12},
   {'start': 20, 'end': 21},
   {'start': 27, 'end': 27},
   {'start': 29, 'end': 30},
   {'start': 32, 'end': 33},
   {'start': 35, 'end': 36},
   {'start': 39, 'end': 39},
   {'start': 41, 'end': 42},
   {'start': 44, 'end': 45},
   {'start': 47, 'end': 48},
   {'start': 50, 'end': 51},
   {'start': 53, 'end': 54},
   {'start': 56, 'end': 57},
   {'start': 59, 'end': 60},
   {'start': 62, 'end': 63},
   {'start': 65, 'end': 66},
   {'start': 68, 'end': 69},
   {'start': 71, 'end': 72},
   {'start': 74, 'end': 75},
   {'start': 77, 'end': 78},
   {'start': 80, 'end': 81},
   {'start': 83, 'end': 84},
   {'start': 86, 'end': 87},
   {'start': 89, 'end': 90},
   {'start': 92, 'end': 93},
   {'start': 95, 'end': 95},
   {'start': 98, 'end': 99},
   {'start': 101, 'end': 102},
   {'star

In [177]:
all_result.keys()

dict_keys(['data/video/How to Increase Code Quality with Clean Code on Android.mp4', 'data/video/Managing Offline Mode in Android Its Easier Than You Think!.mp4', 'data/video/Build Your Own Push Notification Server as an Alternative to Firebase and its Impact on Business.mp4', 'data/video/Simple Steps to Build Blockchain Business Projects.mp4'])

In [179]:
all_faces['data/video/How to Increase Code Quality with Clean Code on Android.mp4']['url']

'https://www.youtube.com/watch?v=UtTPLckT8N0'

In [178]:
all_result['data/video/How to Increase Code Quality with Clean Code on Android.mp4']

[{'start': 26, 'end': 26},
 {'start': 130, 'end': 130},
 {'start': 131, 'end': 131},
 {'start': 11, 'end': 12},
 {'start': 20, 'end': 21},
 {'start': 27, 'end': 27},
 {'start': 29, 'end': 30},
 {'start': 32, 'end': 33},
 {'start': 35, 'end': 36},
 {'start': 39, 'end': 39},
 {'start': 41, 'end': 42},
 {'start': 44, 'end': 45},
 {'start': 47, 'end': 48},
 {'start': 50, 'end': 51},
 {'start': 53, 'end': 54},
 {'start': 56, 'end': 57},
 {'start': 59, 'end': 60},
 {'start': 62, 'end': 63},
 {'start': 65, 'end': 66},
 {'start': 68, 'end': 69},
 {'start': 71, 'end': 72},
 {'start': 74, 'end': 75},
 {'start': 77, 'end': 78},
 {'start': 80, 'end': 81},
 {'start': 83, 'end': 84},
 {'start': 86, 'end': 87},
 {'start': 89, 'end': 90},
 {'start': 92, 'end': 93},
 {'start': 95, 'end': 95},
 {'start': 98, 'end': 99},
 {'start': 101, 'end': 102},
 {'start': 104, 'end': 105},
 {'start': 107, 'end': 108},
 {'start': 110, 'end': 111},
 {'start': 113, 'end': 114},
 {'start': 116, 'end': 117},
 {'start': 1

In [3]:
img = Image.open('sendylenvi regia.png').convert('RGB')
result = find_face_on_data(img)

In [4]:
result_face = find_face_on_data(img)

In [4]:
# result_face['data/video/Developing Learning Culture to Enhance Innovative Work Behavior.mp4']['url']

In [5]:
# result_face['data/video/Developing Learning Culture to Enhance Innovative Work Behavior.mp4']['time']

In [170]:
all_faces['data/video/Developing Learning Culture to Enhance Innovative Work Behavior.mp4']['agg_text']

{'telkom': [{'start': 0, 'end': 24},
  {'start': 28, 'end': 28},
  {'start': 33, 'end': 33},
  {'start': 35, 'end': 35},
  {'start': 40, 'end': 40},
  {'start': 45, 'end': 45},
  {'start': 47, 'end': 47},
  {'start': 52, 'end': 52},
  {'start': 57, 'end': 57},
  {'start': 59, 'end': 59},
  {'start': 62, 'end': 62},
  {'start': 64, 'end': 64},
  {'start': 69, 'end': 69},
  {'start': 74, 'end': 74},
  {'start': 76, 'end': 76},
  {'start': 81, 'end': 81},
  {'start': 86, 'end': 86},
  {'start': 88, 'end': 88},
  {'start': 103, 'end': 103},
  {'start': 105, 'end': 105},
  {'start': 110, 'end': 110},
  {'start': 115, 'end': 115},
  {'start': 117, 'end': 117},
  {'start': 122, 'end': 122},
  {'start': 127, 'end': 127},
  {'start': 129, 'end': 129},
  {'start': 134, 'end': 134},
  {'start': 139, 'end': 139},
  {'start': 141, 'end': 141},
  {'start': 144, 'end': 144},
  {'start': 146, 'end': 146},
  {'start': 151, 'end': 151},
  {'start': 156, 'end': 156},
  {'start': 158, 'end': 158},
  {'sta

In [10]:
all_faces['data/video/Developing Learning Culture to Enhance Innovative Work Behavior.mp4']['agg_text']['inovasi']

[{'start': 73, 'end': 73},
 {'start': 73, 'end': 74},
 {'start': 74, 'end': 76},
 {'start': 78, 'end': 81},
 {'start': 81, 'end': 82},
 {'start': 82, 'end': 82},
 {'start': 266, 'end': 275}]

In [9]:
all_faces['data/video/Developing Learning Culture to Enhance Innovative Work Behavior.mp4']['agg_text'].keys()

dict_keys(['telkom', 'indonesia', '(nwuw uut uema', 'tutlu wrult |euru', 'inn', 'vationday', 'wutju wrult heuru', 'tutju w ult |euru', 'mao', 'wltmw uu r0 ', 'wlh uudaa', 'sendylenvi regia', 'moderator', 'wltiwa uu lava', '5 e n', 'wutlu w ul hena4', 'wuju weruu |euau', 'od', 'statue opubep', 'pyrakid', 'a', '3', 'hatha', 'an', 'husee du', ': 6:', 'eifie', 'colq', '1', '', 'byuoi', 'ncol0ss', 'esg 6 hert', '(tur iutiw uut iaua', 'prhiid', 'fa- ,', 'ma', 'tr', 'ne', 'chathedral', 'ea', '@usee du', '4 f', 'eiffel', '-', 'flata', 'heinc0', 'mardhatillah, ph.d', 'speaker', 'fepid', 'amy', '((ur wtlu wruut uemru', 'eolu', 'fmb', 'praod', 'lowe;', 'fn', 'trua', 'n', "'a", 'musee du', 'elneefn', 'eiftel', 'flaa', 'colo', '((ur wutmwduut kuu', 'eo', 'statueopubea', 'pyramid', ' .', 'm', 'chathedr', 'enn', ': 0.=', 'dpid', 'developing learning culture to enhance innovative work behavior', 'innovation', 'day', 'wlmruu |emru', 'meduederualeurd', 'wmaruu veu u4', 'mdueee  ultlaueu', 'wlw uu hua', 

# Process Audio

In [184]:
all_response_stt = pickle.load(open('all_response_stt','rb'))

In [205]:

def convert_response_stt(data,all_faces):
    return_data = {}
    for i in data.keys():
        i_split = i.split('/')
        i_split[1] = 'video'
        i_split[2] = i_split[2][:-4]+'.mp4'
        key_name = '/'.join(i_split)
        return_data[key_name] = {}
        return_data[key_name]['agg_text'] = {}
        for j in data[i]:
            if len(j[2].strip())>3:
                if j[2].strip() not in return_data[key_name]['agg_text'].keys():
                    return_data[key_name]['agg_text'][j[2].strip()] = [{'start':j[0]//100,
                                                                                 'end':j[1]//100}]
                else:
                    return_data[key_name]['agg_text'][j[2].strip()].append({'start':j[0]//100,
                                                                                 'end':j[1]//100})
        return_data[key_name]['url'] = all_faces[key_name]['url']
    return return_data

In [206]:
debug = convert_response_stt(all_response_stt,all_faces)

In [217]:
retri_stt = Retrieval()
retri_stt.sync(debug)

In [222]:
query_search('design',retri_stt,debug)

{'data/video/What Is Guild and Why Engineering Excellence Matters for World Class Products.mp4': {'time': [{'start': 762,
    'end': 773}],
  'url': 'https://www.youtube.com/watch?v=ao93AQXkdnQ'},
 'data/video/Use API to Accelerate The Making of Your Ideas.mp4': {'time': [{'start': 3784,
    'end': 3800}],
  'url': 'https://www.youtube.com/watch?v=SnsZ6FiKvvs'},
 'data/video/Agile Implementation with Scrum Framework.mp4': {'time': [{'start': 2531,
    'end': 2545}],
  'url': 'https://www.youtube.com/watch?v=EcCqIGcRT_o'},
 'data/video/Crafting Your Powerful Personal Branding.mp4': {'time': [{'start': 1677,
    'end': 1691}],
  'url': 'https://www.youtube.com/watch?v=k7XszijoXjY'},
 'data/video/Fast and Furious in UX Design.mp4': {'time': [{'start': 1594,
    'end': 1606},
   {'start': 4591, 'end': 4598},
   {'start': 2345, 'end': 2354},
   {'start': 1910, 'end': 1920},
   {'start': 4473, 'end': 4488},
   {'start': 5616, 'end': 5635},
   {'start': 1876, 'end': 1890}],
  'url': 'https://

In [186]:
all_response_stt['data/audio/Design Thinking for Career Planning and Development.wav']

[(264, 1117, ''),
 (1764, 2184, ''),
 (2311, 2342, ''),
 (2475, 2877, 'dan '),
 (3229, 3618, ''),
 (4266, 4684, ''),
 (4818, 4841, ''),
 (4972, 5399, 'oh '),
 (5763, 6121, 'datang '),
 (7046,
  9357,
  'assalamualaikum wabarakatu semangat pagi pagi pagi pagi bersyukur lagi di acara apa kabar semua yang ada di kantor rumah atau dimana manapun kalian berada semoga saat selalu ya dan tetap produktif pastinya di masa sistem seperti saat ini '),
 (9428,
  10831,
  'perkenalkan saya tanya dari yang akanndu acara dari hari ini topik kali ini udah kalah serunyaan nih dengan topik topik sebelumnya kali ini kita akan membahas tentanggaring '),
 (10916,
  13943,
  'karena pada dunia pekerjaan baik ketika merencanakan maupun dalam perjalanan karir seringkali kita di dihadapkan dengan berbagai pilihan saat mengambil keputusan yang lahir dan banyak digunakan orang khususnya pada desainer dalam berininovasi dan mengembangkan produk ternyata juga memiliki teguran yangseal yang lain salah satunya dapat

In [ ]:
import io
import re
import glob
import nltk
import numpy as np
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer 
nltk.download('stopwords')

def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.dot(a, a.T)*np.dot(b.T, b))
    return cos_sim
def cosine_sim_batch(a,b):
    similarity = a * b
    similarity = similarity.toarray()
    return similarity
class Retrieval :
    def __init__(self) :
        self.stop_words = set(stopwords.words('indonesian')) 
        self.stop_words.add('telkom')
        self.stop_words.add('indonesia')
                
    def retrieve(self, query, n, ngram=1):
        if ngram == 2 :
            df = self.retrieve_bigram(query)
        else :
            df = self.retrieve_unigram(query)
        
#         files = get_files()
#         df.docs = df.docs.str.replace('.txt', '.pdf')

#         df = pd.merge(df, files, on='docs')
        
        df = df.head(n)
        df['file'] = df.docs.apply(lambda x: x.split('_&_')[0])
        df['words'] = df.docs.apply(lambda x: x.split('_&_')[1])
        df = df.drop('docs',axis=1)
        return df
    
    def retrieve_unigram(self, query) :
        query_vector_unigram = self.unigram.transform([query])
        
        scores_unigram = cosine_sim_batch(self.mentahan_tfidf_unigram,query_vector_unigram.T).ravel()
                
        self.result_unigram = scores_unigram
        df = pd.DataFrame({'docs':self.vectors_unigram.keys(), 'score':scores_unigram})
        df = df.sort_values('score', ascending=False).reset_index(drop=True)
        return df
    
    def retrieve_bigram(self, query) :
        query_vector_bigram = self.bigram.transform([query])
        scores_bigram = []
        for f in self.vectors_bigram :
            a = self.vectors_bigram[f]
            b = query_vector_bigram.T
            s = cosine_sim(a, b)
            scores_bigram.append(s.A1[0])
        self.result_bigram = scores_bigram
        df = pd.DataFrame({'docs':selfself.vectors_bigram.keys(), 'score':scores_bigram})
        df = df.sort_values('score', ascending=False).reset_index(drop=True)
        return df

    def retrieve_unigram_depricated(self, query) :
        query_vector_unigram = self.unigram.transform([query])
        scores_unigram = []
        for f in self.vectors_unigram :
            a = self.vectors_unigram[f]
            b = query_vector_unigram.T
            s = cosine_sim(a, b)
            scores_unigram.append(s.A1[0])
                
        self.result_bigram = scores_unigram
        df = pd.DataFrame({'docs':self.vectors_unigram.keys(), 'score':scores_unigram})
        df = df.sort_values('score', ascending=False).reset_index(drop=True)
        return df
    
    def retrieve_bigram_depricated(self, query) :
        query_vector_bigram = self.bigram.transform([query])
        scores_bigram = []
        for f in self.vectors_bigram :
            a = self.vectors_bigram[f]
            b = query_vector_bigram.T
            s = cosine_sim(a, b)
            scores_bigram.append(s.A1[0])
        self.result_bigram = scores_bigram
        df = pd.DataFrame({'docs':selfself.vectors_bigram.keys(), 'score':scores_bigram})
        df = df.sort_values('score', ascending=False).reset_index(drop=True)
        return df

    def sync(self, all_data) :
        bigram=TfidfVectorizer(use_idf=True, stop_words=self.stop_words, ngram_range=(1, 2))
        unigram=TfidfVectorizer(use_idf=True, stop_words=self.stop_words)
        symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
        lines = {}
        list_sentence = []
        
        for f in all_data.keys():
            for sentence in all_data[f]['agg_text'].keys():
                lines[f+'_&_'+sentence] = sentence
                list_sentence.append(sentence)
                
        tfidf_bigram = bigram.fit_transform(lines.values())
        tfidf_unigram = unigram.fit_transform(lines.values())                
        
        self.mentahan_tfidf_unigram = tfidf_unigram
        self.mentahan_tfidf_bigram = tfidf_bigram

        vectors_bigram = {f:v for f, v in zip(lines.keys(), tfidf_bigram)}
        vectors_unigram = {f:v for f, v in zip(lines.keys(), tfidf_unigram)}
        
        self.list_sentence = list_sentence
        self.bigram = bigram
        self.unigram = unigram
        self.vectors_bigram = vectors_bigram
        self.vectors_unigram = vectors_unigram
        